<a href="https://colab.research.google.com/github/BrockH3/Intro_to_deep_Learning/blob/main/Homework_1_Hunter_Brock_801179909.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
train = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size = 100 )

val = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transforms.ToTensor())
valloader = torch.utils.data.DataLoader(val, shuffle=False, batch_size = 100)



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
#classifier model
class CIFAR10classifier(nn.Module):
  def __init__(self):
    #3072 inputs, 3 fully connected hidden layers, 10 outputs
    super(CIFAR10classifier, self).__init__()
    self.fc1 = nn.Linear(3072,1000)
    self.fc2 = nn.Linear(1000, 500)
    self.fc3 = nn.Linear(500, 50)
    self.out = nn.Linear(50,10)

    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    x = self.relu(x)
    x = self.fc3(x)
    x = self.relu(x)
    x = self.out(x)
    return x

#initialize model
model = CIFAR10classifier()

In [ ]:
#loss function
criterion = nn.CrossEntropyLoss()
#optimizer
optimizer = optim.SGD(model.parameters(), lr = .01)

#training loop
epochs = 20
for epoch in range(epochs):
  model.train()
  for inputs, targets in trainloader:
    optimizer.zero_grad()
    batch_size = inputs.shape[0]
    outputs = model(inputs.view(batch_size,-1))
    loss = criterion(outputs,targets)
    loss.backward()
    optimizer.step()
  if (epoch%5 ==0):
    print('epoch: ', epoch, 'loss: ', loss)


epoch:  0 loss:  tensor(2.1599, grad_fn=<NllLossBackward0>)
epoch:  5 loss:  tensor(1.7604, grad_fn=<NllLossBackward0>)
epoch:  10 loss:  tensor(1.7615, grad_fn=<NllLossBackward0>)
epoch:  15 loss:  tensor(1.4423, grad_fn=<NllLossBackward0>)


In [ ]:
print('loss: ', float(loss))


loss:  1.5902841091156006


In [ ]:
#validation and scores
model.eval()
correct = 0
total = 0
all_predictions = []
all_targets = []
with torch.no_grad():
    for inputs, targets in valloader:
      batch_size = inputs.shape[0]
      outputs = model(inputs.view(batch_size,-1))
      _, predicted = torch.max(outputs, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()
      all_predictions.extend(predicted.cpu().numpy())
      all_targets.extend(targets.cpu().numpy())


In [ ]:
accuracy = correct/total
precision = precision_score(all_targets,all_predictions, average = 'macro')
recall = recall_score(all_targets,all_predictions, average = 'macro')
f1 = f1_score(all_targets,all_predictions, average = 'macro')
decision_matrix = confusion_matrix(all_targets, all_predictions)

print('accuracy: ', accuracy, ' recall: ', recall, ' f1 sore: ', f1)

accuracy:  0.472  recall:  0.47199999999999986  f1 sore:  0.46712920833910526


In [ ]:
print(decision_matrix)

[[454  35  58  89  15  37  37  63 111 101]
 [ 19 587   4  49   6  26  16  38  31 224]
 [ 58  28 231 180  66  96 201  99  19  22]
 [ 10  16  38 466  14 194 134  61  11  56]
 [ 36  14 119 123 235  82 220 127  16  28]
 [  6  13  55 292  16 376 101  97  18  26]
 [  2   9  29 175  46  54 629  29   7  20]
 [ 14  16  34  93  36  89  53 585   6  74]
 [100  87   8  67  16  35  14  17 512 144]
 [ 18 151   2  62   5  16  32  45  24 645]]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filepath = '/content/drive/My Drive/Intro to deep learning/datasets/house-train.csv'

house_ds = pd.read_csv(filepath)
y = house_ds.pop('SalePrice')
x = house_ds

columns = ['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF'
                , 'FullBath', 'YearBuilt', 'YearRemodAdd', 'Fireplaces'
                ,'LotFrontage','WoodDeckSF','OpenPorchSF'
                ,'ExterQual','Neighborhood','MSZoning'
                ,'Alley','LotShape','LandContour','Condition1','HouseStyle','MasVnrType','SaleCondition']

x = x[columns]
# Remove Nulls
## GarageArea in test data
x['GarageArea'] = x['GarageArea'].fillna(x['GarageArea'].mean())

x['TotalBsmtSF'] = x['TotalBsmtSF'].fillna(x['TotalBsmtSF'].mean())


#split validation and training sets
np.random.seed(0)
x_train, x_val,y_train,y_val = train_test_split(x, y, train_size = .8, test_size = .2, random_state = 0)
x_train = torch.tensor(x_train, dtype=torch.float32)
x_val = torch.tensor(x_val, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

#create datasets and dataloaders
train_dataset = TensorDataset(x_train,y_train)
val_dataset = TensorDataset(x_val,y_val)
train_loader = DataLoader(dataset=train_dataset,shuffle=True)
val_loader = DataLoader(dataset=val_dataset,shuffle=True)

<ipython-input-13-0a4a0179299b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['GarageArea'] = x['GarageArea'].fillna(x['GarageArea'].mean())
<ipython-input-13-0a4a0179299b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['TotalBsmtSF'] = x['TotalBsmtSF'].fillna(x['TotalBsmtSF'].mean())


ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
#create model for regression, 2 hidden layers, 21 inputs 1 output, relu activation function
class reg(nn.Module):
  def __init__(self):
    super(reg,self).__init__()
    self.fc1 = nn.Linear(21,100)
    self.fc2 = nn.Linear(100,20)
    self.fc3 = nn.Linear(20,1)

  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.relu(self.fc3(x))
    return x

In [ ]:
#regression training loop, (influenced by github code)
model = reg()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01)

train_loss = 0
train_loss_epochs = []
val_loss_epochs = []

epochs = 50

for epoch in range(epochs):
  model.train()
  for inputs,targets in train_loader:
    optimizer.zero_grad()
    out = model(inputs)
    loss = criterion(out,targets)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*inputs.size(0)
  train_loss/=len(train_loader.dataset)
  train_loss_epochs.append(train_loss)
  val_loss = 0
  val_total = 0

  with torch.no_grad():
    for inputs,targets in val_loader:
      out = model(val_loader)
      loss = criterion(out,targets)
      val_loss += loss.item()*inputs.size(0)
      val_total += ((out-targets)**2).sum().item()

  val_loss/=len(val_loader.dataset)
  val_loss_epochs.append(val_loss)

  rmse = np.sqrt(val_total/len(val_loader.dataset))

print('train loss: ', train_loss, ' val_loss: ', val_loss, ' RMSE: ', rmse)



